In [1]:
import numpy as np
import os
import pysynphot as s

In [2]:
jpas_filters_path = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/JPAS_filters'
test_specs        = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/Specs'
jpas_filters_list = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/jpas_filters_list.txt', dtype=str)
specs_list        = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/specslist.txt', dtype=str)

In [3]:
# Constants --------------------------------------------------------------------------------------------------------
c = 2.99792458E18                  # Light Speed in Angstrom s^-1

In [4]:
# Setting the T80 M1 effective area in cm^2
s.refs.PRIMARY_AREA = 4400


In [5]:
for each_spectrum in specs_list:
    # Transforming the specs from f_lambda to f_nu -----------------------------------------------------------------
    wavelength = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[0])
    f_lambda   = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[1])
    for jpas_filters in jpas_filters_list:
        filter_name          = jpas_filters.split('.')[0]
        jpas_filter_bandpass = np.array(np.loadtxt(os.path.join(jpas_filters_path, jpas_filters)))
        sdss_spectrum        = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=(0,1))
        jpas_filter_bandpass = s.FileBandpass(jpas_filter_bandpass)
#         print sdss_spectrum
#         exit()
#         sdss_spectrum        = s.FileSpectrum(sdss_spectrum)
#         photometry           = s.Observation(sdss_spectrum, jpas_filter_bandpass, binset=np.arange(3000, 11000, 1))

/home/mldantas/miniconda2/lib/python2.7/posixpath.py:333: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if path == '':


AttributeError: 'numpy.ndarray' object has no attribute 'startswith'